# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

dom=cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)
dom_blur=cv2.GaussianBlur(dom,(5,5),0)
dom_blur_lapl=cv2.Laplacian(dom_blur,cv2.CV_32F)

def crossing(LoG,thr):
    x,y=LoG.shape
    result=np.ones(LoG.shape)
    for k in range(1,x-1):
        for m in range(1,y-1):
            otoczenie=LoG[k-1:k+2,m-1:m+2]
            max_otoczenie=otoczenie.max()
            min_otoczenie=otoczenie.min()
            if min_otoczenie*max_otoczenie<0:
                if LoG[k,m]>0:
                     result[k,m]=LoG[k,m]+np.abs(min_otoczenie)
                if LoG[k,m]<0:
                     result[k,m]=np.abs(LoG[k,m])+max_otoczenie
                
    new_result=cv2.normalize(result,None,0,255,cv2.NORM_MINMAX)
    Result=np.where(new_result < thr, 0, 1)
    result_scale=Result*255
    resultn=result_scale.astype('uint8')
    final_result=cv2.medianBlur(resultn,5)
    return final_result

dom_crossing=crossing(dom_blur_lapl,3)
plt.imshow(dom_crossing,cmap='gray',vmin=0,vmax=256)
plt.axis('off')
plt.show()

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w~1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj pierwszą część algorytmu detekcji krawędzi Canny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
7. Na jednym obrazie zaznacz piksele należące do obrazu $g_{NH}$ jako czerwone oraz należące do obrazu $g_{NL}$ jako niebieskie.
Wyświetl obraz.

In [ ]:
def canny(image,Th,Tl):
    image_blur=cv2.GaussianBlur(image,(3,3),0)
    M1=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    M2=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    Gx=cv2.filter2D(image_blur, -1, M1)
    Gy=cv2.filter2D(image_blur, -1, M2)
    M=np.sqrt(pow(Gx,2)+pow(Gy,2))
    alpha=np.arctan2(Gy,Gx)
    alpha_degrees=(alpha*180)/np.pi
    alpha_degrees[alpha_degrees < 0] =alpha_degrees[alpha_degrees < 0]+180
    direction=np.zeros(image.shape)
    d1=1
    d2=2
    d3=3
    d4=4
    for k in range(image.shape[0]):
        for m in range(image.shape[1]):
            #1
            if (0<alpha_degrees[k,m]<22.5) or (157.5 <= alpha_degrees[k,m] <= 180):
                direction[k,m]=d1
            #2
            elif 22.5<=alpha_degrees[k,m]<67.5:
                direction[k,m]=d2
            #3
            elif 67.5<=alpha_degrees[k,m]<112.5:
                direction[k,m]=d3
            #4
            elif 112.5<=alpha_degrees[k,m]<157.5:
                direction[k,m]=d4
    Gn=nonmax(direction,M)
    Ghn=Gn>=Th
    Gnl=np.where(np.logical_and(Th>Gn,Gn>=Tl),1,0)
    return Ghn,Gnl

In [ ]:
def nonmax(direction,M,d1=1,d2=2,d3=3,d4=4):
    result=np.zeros(direction.shape)
    for k in range(1,direction.shape[0]-1):
        for m in range(1,direction.shape[1]-1):
            if direction[k,m]==d1:
                if M[k,m-1]>M[k,m] or M[k,m+1]>M[k,m]:
                    result[k,m]=0
                else:
                    result[k,m]=M[k,m]
            elif direction[k,m]==d2:
                if M[k+1,m-1]>M[k,m] or M[k-1,m+1]>M[k,m]:
                    result[k,m]=0
                else:
                    result[k,m]=M[k,m]            
            elif direction[k,m]==d3:
                if M[k+1,m]>M[k,m] or M[k-1,m]>M[k,m]:
                    result[k,m]=0
                else:
                    result[k,m]=M[k,m]
            elif direction[k,m]==d4:
                if M[k-1,m-1]>M[k,m] or M[k+1,m+1]>M[k,m]:
                    result[k,m]=0
                else:
                    result[k,m]=M[k,m]

    return result
    

In [ ]:
dom=cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)
result=canny(dom,12,6)
x,y=result[1].shape
result_image=np.zeros((x,y,3))
for k in range(x):
  for m in range(y):
    if result[1][k,m]==1:
      result_image[k,m]=[0,0,255]
    if result[0][k,m]==1:
      result_image[k,m]=[255,0,0]
plt.imshow(result_image)
plt.axis('off')
plt.title('Gnh-czerwony,Gnl-niebieski')

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z wykonaną częściową implementacją w poprzednim ćwiczeniu.
Na czym polegają różnice?

In [ ]:
dom=cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)
result=canny(dom,12,6)
dom_canny=cv2.Canny(dom,6,12,None,3,1)
fig,ax=plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(dom_canny,cmap='gray')
ax[0].axis('off')
ax[0].set_title('funkcja z biblioteki')
ax[1].imshow((result[0]+result[1]),cmap='gray')
ax[1].axis('off')
ax[1].set_title('zaimplementowana funkcja')

#zaimpelmentowan  funkcja canny poradziła sobie lepiej z tłem,ale ta z biblioteki zdecydowanie lepiej z obiektem 